In [1]:
import json
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
fs = pd.read_json('../montreal_subset/featureset1.json')

In [3]:
fs.head()

,categories_avg_rating,categories_biz_count,categories_reviews,cool,elite_user,funny,months_yelping,std_dev_rating,total_reviews,useful,user_id
0,4.0,2,2,0,elite,0,28,0.0,6,1,Vi6yC7XiZDADYG73Z0MJAQ
1,4.0,2,2,0,elite,0,28,0.0,6,1,Vi6yC7XiZDADYG73Z0MJAQ
10,4.0,2,2,0,elite,0,28,0.0,6,1,Vi6yC7XiZDADYG73Z0MJAQ
100,3.0,1,1,1,not_elite,0,64,0.0,4,2,CsORgjfkar9ozMlOI1nRkQ
101,3.5,2,2,0,elite,0,26,0.5,5,0,0qtp4spComZB71ci0qG-bg


In [4]:
fs['is_train'] = np.random.uniform(0,1,len(fs)) <= .75

In [5]:
train, test = fs[fs['is_train']==True], fs[fs['is_train']==False]

In [6]:
features = pd.Index([
           'categories_avg_rating',
           'categories_biz_count',
           'categories_reviews',
           'cool',
           'funny',
           'months_yelping',
           'std_dev_rating',
           'total_reviews',
           'useful'
          ])


In [7]:
clf = RandomForestClassifier(n_jobs=2)
y, _ = pd.factorize(train['elite_user'])
clf.fit(train[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
target_names = np.array(["not_elite", "elite"])

In [9]:
preds = target_names[clf.predict(test[features])]

In [10]:
pd.crosstab(test['elite_user'], preds, rownames=['actual'], colnames=['preds'])

preds,elite,not_elite
actual,,
elite,2,124
not_elite,67,7


In [11]:
from sklearn import metrics

In [12]:
print metrics.classification_report(test['elite_user'], preds)

             precision    recall  f1-score   support

      elite       0.03      0.02      0.02       126
  not_elite       0.05      0.09      0.07        74

avg / total       0.04      0.04      0.04       200

